In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import progressbar as progressbar

%load_ext autoreload
%autoreload 2

In [2]:
print(f'TF version: {tf.__version__}')
tf.config.list_physical_devices()

TF version: 2.1.0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Read Data

In [3]:
# path = 'C:/Users/robin.opdam/Google Drive/Thesis (Msc)/Thesis_shared_files/'
# path = '/Users/Robin/Google Drive/Thesis (Msc)/Thesis_shared_files/'
path = '../datasets/'

## Amazon Fashion

In [4]:
# data_path = 'data/Amazon/'
# file_name = 'Amazon_full' # file_name = 'Amazon_05_users' 
# file_name = 'Amazon_01_users'
# file_name = 'am_80k_users'
file_name = 'am_40k_users'

## MovieLens

In [5]:
# data_path = 'data/ML/'
# file_name = 'ml_1m'
# file_name = 'ML_full' # file_name = 'ML_05_users'
# file_name = 'ML_01_users'

In [6]:
df = pd.read_pickle(path + file_name)
df.user_id = df.user_id.astype('category').cat.codes
df.item_id = df.item_id.astype('category').cat.codes
df.head()

,user,item,datetime,rating,user_id,item_id
1845392,AGYA6NBNU1VUH,B0020B4EYU,2015-04-14,3.0,34371,11002
919114,AGYA6NBNU1VUH,B0000ZFDD4,2015-04-14,3.0,34371,179
5872323,AGYA6NBNU1VUH,B00KR6I6IK,2015-04-14,5.0,34371,75683
1237107,AGYA6NBNU1VUH,B0000ZFDD4,2015-04-14,3.0,34371,179
9971633,AGYA6NBNU1VUH,B008GXBRNM,2015-12-20,5.0,34371,35964


### Create ML split (paper)

In [ ]:
test_set = pd.DataFrame()
train_set = pd.DataFrame()
pbar = progressbar.ProgressBar()
for u in pbar(df.user_id.unique()):
    last_item = df[df.user_id==u].iloc[-1:]
    test_set = test_set.append(last_item)
    remaining = df[df.user_id==u].iloc[:-1]
    train_set = pd.concat([train_set, remaining])

In [7]:
test_set = pd.read_pickle(path + 'ml_1m_test')
train_set = pd.read_pickle(path + 'ml_1m_train')

---
# Data Prep

## Dataset Params

In [7]:
val_perc = test_perc = 0.1
n_last_items_val = n_last_items_test = 1

total_items = len(df.item_id.unique())
total_users = len(df.user_id.unique())

In [10]:
len(train_set.user_id.unique())

40000

## Create Split

In [9]:
from Data_prep import train_val_test_split
datasets = train_val_test_split(df, val_perc, test_perc, n_last_items_val, n_last_items_test)
train_set, val_set, test_set = datasets

---
# Neural Collaborative Filtering (NCF)
Using the NCF framework we build Generalized Matrix Factorisation (GMF), Multiplayer Perceptron Matrix Factorisation (MLP) and combine the two in Neural Matrix Factorisation (NeuMF)
- paper: http://papers.www2017.com.au.s3-website-ap-southeast-2.amazonaws.com/proceedings/p173.pdf
- blog: https://medium.com/@victorkohler/collaborative-filtering-using-deep-neural-networks-in-tensorflow-96e5d41a39a1
- code: https://github.com/Leavingseason/NeuralCF/blob/master

## Set Parameters

In [45]:
run_num = 'am_40k_GS_nolf_8'
# run_num = 'ml_1m_nolf_8'

In [46]:
GMF_params = {
    'learning_rate': 0.001,
    'batch_size': 256,
    'nolf': 8,
    'regs': [0.000001,0.000001], #ML: [0.000001,0.000001]
    'epochs': 20,
    'sample_size': len(train_set),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'ckpt_dir': f'../NeuMF_storage/GMF_ckpts_{run_num}/ckpts',
    'optimizer':'Adam'
}

In [47]:
# MLP_params = {
#     'learning_rate': 0.001,
#     'batch_size': 256,
#     'layers': [16,32,16,8],
#     'reg_layers': [0.0001,0,0,0],   #ML: [+-0.0005,0,0,0],
#     'epochs': 20,
#     'sample_size': len(train_set),#int(0.5*len(train_set.user_id.unique())),
#     'num_neg': 4,
#     'ckpt_dir': f'../NeuMF_storage/MLP_ckpts_{run_num}/ckpts',
#     'optimizer':'Adam'
# }

In [48]:
NeuMF_params = {
    'learning_rate': 0.001,
    'batch_size': 256,
    'layers': [16,32,16,8],
    'reg_layers': [0.0001,0,0,0],
    'reg_mf': [0.000001,0.000001],
    'nolf': 8,
    'epochs': 2,
    'sample_size': len(train_set),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'ckpt_dir': f'../NeuMF_storage/NeuMF_ckpts_{run_num}/ckpts',
    'optimizer':'Adam'
}

## Load samples

In [49]:
sample_path = '../NeuMF_storage/samples_ml_1m/'
sample_name = 'ml_1m_sample_'

sample_path = '../NeuMF_storage/samples_am_40k/'
sample_name = 'am_40k_sample_'

sample_path = '../NeuMF_storage/samples_am_40k_new/'
sample_name = 'am_40k_sample_'

In [16]:
%%time
import csv
samples = []
pbar = progressbar.ProgressBar()
for sample_num in pbar(range(20)):
    with open(f'{sample_path}{sample_name}{sample_num}.csv', 'r') as read_obj:
        sample = [[], [], []]
        csv_reader = csv.reader(read_obj)
        sample_str = list(csv_reader)
        
        for user_input, item_input, label in zip(sample_str[0], sample_str[1], sample_str[2]):
            sample[0].append(int(user_input))
            sample[1].append(int(item_input))
            sample[2].append(int(label))
        
    samples.append(sample)

CPU times: user 39.9 s, sys: 3.58 s, total: 43.4 s
Wall time: 43.6 s


---
# Grid Search: NeuMF

## Init

In [88]:
from NCF import NCF
NCF = NCF(total_users, total_items, NeuMF_params=NeuMF_params)
NCF.build_NeuMF_model()

## Parameter Space

In [89]:
learning_rates = [0.0005, 0.0001]
batch_sizes = [256]
gmf_regs = [[0,0], [0.0000001, 0.0000001], [0.000001, 0.000001], [0.00001, 0.00001]]
mlp_regs = [[0,0,0,0],
            [0.0000001, 0.0000001, 0.0000001, 0.0000001],
            [0.000001, 0.000001, 0.000001, 0.000001], 
            [0.00001, 0.00001, 0.00001, 0.00001], 
            [0.0001, 0.0001, 0.0001, 0.0001]]#,
#             [0.001, 0.001, 0.001, 0.001],
#             [0.000001, 0, 0, 0],
#             [0.00001, 0, 0, 0],
#             [0.0001, 0, 0, 0],
#             [0, 0, 0, 0.0001],
#             [0, 0, 0, 0.00001],
#             [0, 0, 0, 0.000001]]
            
total_runs = len(learning_rates) * len(batch_sizes) * len(gmf_regs) * len(mlp_regs)
total_runs

40

### Init eval params

In [90]:
from Evaluation import get_metrics
steps = 5
rank_at = 20
sample_len = 100

## Init store results

In [91]:
store_path = '../NeuMF_storage/GS/'
gs_result_name = f'GS_NeuMF_bs_{batch_sizes[0]}'
storage_columns = ['test_rec@10', '100_sample_val_rec@10', 'val_metrics', 'test_metrics', 'loss', 'train_time'] + list(NeuMF_params.keys())

In [92]:
gs_results = pd.DataFrame(columns=storage_columns)
gs_results.to_pickle(store_path + gs_result_name)

## Run Grid Search

In [93]:
import time
NCF.NeuMF_params['epochs'] = 14

run = 0
model_name = 'NeuMF'

for learning_rate in learning_rates:
    NCF.NeuMF_params['learning_rate'] = learning_rate
    
    for batch_size in batch_sizes:
        NCF.NeuMF_params['batch_size'] = batch_size
        
        for gmf_reg in gmf_regs:
            NCF.NeuMF_params['reg_mf'] = gmf_reg
            
            for mlp_reg in mlp_regs:
                s = time.time()
                NCF.NeuMF_params['reg_layers'] = mlp_reg
                print('\n', '='*50)
                print(f'run {run}/{total_runs}')
                NCF.build_NeuMF_model()

                val_metrics = NCF.train_model(model_name, samples, train_set, val_set.iloc[:100], verbose=2) 
                t = time.time() - s
                ranked_df = NCF.get_predictions(model_name, test_set)
                test_metrics = get_metrics(ranked_df, steps, rank_at, stats=False)
                val_recall = val_metrics[-1:][0]['recall'].iloc[2]
                
                results = dict(NCF.NeuMF_params,
                    **{'test_rec@10':test_metrics['recall'].iloc[2], 
                    '100_sample_val_rec@10': val_recall,
                    'val_metrics':val_metrics,
                    'test_metrics':test_metrics,
                    'loss':NCF.history[model_name]['loss'],
                    'train_time':t})

                pd.read_pickle(store_path + gs_result_name).append(results, ignore_index=True).to_pickle(store_path + gs_result_name)

                run += 1       


run 0/40


100% |########################################################################|



run 1/40


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
100% |########################################################################|



run 2/40


...

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
100% |########################################################################|



run 38/40


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
100% |########################################################################|



run 39/40


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
100% |########################################################################|


In [94]:
pd.read_pickle(store_path + gs_result_name).sort_values('test_rec@10', ascending=False)

,test_rec@10,100_sample_val_rec@10,val_metrics,test_metrics,loss,train_time,learning_rate,batch_size,layers,reg_layers,reg_mf,nolf,epochs,sample_size,num_neg,ckpt_dir,optimizer
22,0.01925,0.19,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.46384, 0.35846, 0.325, 0.30419, 0.28889, 0....",1294.881698,0.0001,256,"[16, 32, 16, 8]","[1e-06, 1e-06, 1e-06, 1e-06]","[0, 0]",8,14,343771,4,../NeuMF_storage/NeuMF_ckpts_am_40k_GS_nolf_8/...,Adam
20,0.01750,0.19,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.46905, 0.37478, 0.33356, 0.31043, 0.29341, ...",1531.528490,0.0001,256,"[16, 32, 16, 8]","[0, 0, 0, 0]","[0, 0]",8,14,343771,4,../NeuMF_storage/NeuMF_ckpts_am_40k_GS_nolf_8/...,Adam
24,0.00500,0.18,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.4719, 0.34554, 0.33859, 0.33644, 0.33247, 0...",1309.244294,0.0001,256,"[16, 32, 16, 8]","[0.0001, 0.0001, 0.0001, 0.0001]","[0, 0]",8,14,343771,4,../NeuMF_storage/NeuMF_ckpts_am_40k_GS_nolf_8/...,Adam
23,0.00425,0.15,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.44882, 0.32616, 0.30709, 0.29683, 0.28878, ...",1301.970975,0.0001,256,"[16, 32, 16, 8]","[1e-05, 1e-05, 1e-05, 1e-05]","[0, 0]",8,14,343771,4,../NeuMF_storage/NeuMF_ckpts_am_40k_GS_nolf_8/...,Adam
32,0.00375,0.17,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.45936, 0.33743, 0.30245, 0.28642, 0.27368, ...",1292.874761,0.0001,256,"[16, 32, 16, 8]","[1e-06, 1e-06, 1e-06, 1e-06]","[1e-06, 1e-06]",8,14,343771,4,../NeuMF_storage/NeuMF_ckpts_am_40k_GS_nolf_8/...,Adam
29,0.00375,0.18,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.46322, 0.34414, 0.33903, 0.33794, 0.33611, ...",1176.265156,0.0001,256,"[16, 32, 16, 8]","[0.0001, 0.0001, 0.0001, 0.0001]","[1e-07, 1e-07]",8,14,343771,4,../NeuMF_storage/NeuMF_ckpts_am_40k_GS_nolf_8/...,Adam
27,0.00375,0.14,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.46073, 0.33911, 0.30314, 0.28605, 0.27259, ...",1174.188863,0.0001,256,"[16, 32, 16, 8]","[1e-06, 1e-06, 1e-06, 1e-06]","[1e-07, 1e-07]",8,14,343771,4,../NeuMF_storage/NeuMF_ckpts_am_40k_GS_nolf_8/...,Adam
34,0.00350,0.17,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.48317, 0.35098, 0.34229, 0.34012, 0.3387, 0...",1187.408793,0.0001,256,"[16, 32, 16, 8]","[0.0001, 0.0001, 0.0001, 0.0001]","[1e-06, 1e-06]",8,14,343771,4,../NeuMF_storage/NeuMF_ckpts_am_40k_GS_nolf_8/...,Adam
39,0.00325,0.17,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.49545, 0.34924, 0.34031, 0.33898, 0.33764, ...",1175.592839,0.0001,256,"[16, 32, 16, 8]","[0.0001, 0.0001, 0.0001, 0.0001]","[1e-05, 1e-05]",8,14,343771,4,../NeuMF_storage/NeuMF_ckpts_am_40k_GS_nolf_8/...,Adam
31,0.00300,0.16,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.46988, 0.35329, 0.31248, 0.29199, 0.27796, ...",1178.197947,0.0001,256,"[16, 32, 16, 8]","[1e-07, 1e-07, 1e-07, 1e-07]","[1e-06, 1e-06]",8,14,343771,4,../NeuMF_storage/NeuMF_ckpts_am_40k_GS_nolf_8/...,Adam


---
## NeuMF

### Full set scores

In [ ]:
%%time
NeuMF_ranked_df_full = NCF.get_predictions('NeuMF', test_set)

In [ ]:
from Evaluation import get_metrics
get_metrics(NeuMF_ranked_df_full, steps, rank_at, stats=False)

In [40]:
metrics_all_sample.to_pickle('../results/NCF/NCF_metrics_all_sample_am_40k')

### Sample scores

In [ ]:
%%time
NeuMF_ranked_df = NCF.sample_prediction('NeuMF', train_set, test_set)

In [ ]:
from Evaluation import get_metrics
get_metrics(NeuMF_ranked_df, steps, rank_at, stats=False)

---
# Grid Search: GMF

## Init

In [15]:
from NCF import NCF
NCF = NCF(total_users, total_items, GMF_params=GMF_params)
NCF.build_GMF_model()

# NCF.GMF_params['epochs'] = 10

## Parameter Space

In [16]:
learning_rates = [0.005, 0.001, 0.0005, 0.0001]
batch_sizes = [128, 256]
regs = [[0.001, 0.001], [0.0001, 0.0001], [0.00001,0.00001], [0.000001,0.000001]]
            
total_runs = len(learning_rates) * len(batch_sizes) * len(regs)
total_runs

32

### Init eval params

In [17]:
from Evaluation import get_metrics
steps = 5
rank_at = 20
sample_len = 100

## Init store results

In [3]:
store_path = '../NeuMF_storage/GS/'
gs_result_name = 'GS_GMF_with_reg_bs_128_256'
storage_columns = ['test_rec@10', 'val_metrics', 'test_metrics', 'loss'] + list(GMF_params.keys()

In [20]:
gs_results = pd.DataFrame(columns=storage_columns)
gs_results.to_pickle(store_path + gs_result_name)

## Run Grid Search

In [21]:
from Evaluation import get_metrics

run = 0
for learning_rate in learning_rates:
    NCF.GMF_params['learning_rate'] = learning_rate
    
    for batch_size in batch_sizes:
        NCF.GMF_params['batch_size'] = batch_size
        
        for reg in regs:
            print('\n', '='*50)
            print(f'run {run}/{total_runs}')
            NCF.GMF_params['regs'] = reg
            NCF.build_GMF_model()
            
            val_metrics = NCF.train_model('GMF', samples, train_set, test_set.iloc[:500], verbose=2) 
            ranked_df = NCF.sample_prediction('GMF', train_set, test_set)
            test_metrics = get_metrics(ranked_df, steps, rank_at, stats=False)
            results = dict(NCF.GMF_params,
                **{'test_rec@10':test_metrics['recall'].iloc[2], 
                'val_metrics':val_metrics,
                'test_metrics':test_metrics,
                'loss':NCF.history['GMF']['loss']})

            pd.read_pickle(store_path + gs_result_name).append(results, ignore_index=True).to_pickle(store_path + gs_result_name)
            
            run += 1      


run 0/32
Train on 4970845 samples


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


4970845/4970845 - 125s - loss: 0.5010


100% |########################################################################|


Train on 4970845 samples
4970845/4970845 - 124s - loss: 0.5004
Train on 4970845 samples
4970845/4970845 - 124s - loss: 0.5004


...

100% |########################################################################|


Train on 4970845 samples
4970845/4970845 - 63s - loss: 0.2778
Train on 4970845 samples
4970845/4970845 - 63s - loss: 0.2563


100% |########################################################################|


Train on 4970845 samples
4970845/4970845 - 63s - loss: 0.2380


100% |########################################################################|


In [4]:
pd.read_pickle(store_path + 'all_GMF_GS')

,test_rec@10,val_metrics,test_metrics,loss,learning_rate,batch_size,nolf,regs,epochs,sample_size,num_neg,ckpt_dir,optimizer
8,0.001490,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.25461, 0.17192, 0.15172, 0.14021, 0.13227, ...",0.0005,512,8,"[0.001, 0.001]",10,994169,4,../NeuMF_storage/GMF_ckpts_ml_1m_nolf_8/ckpts,Adam
12,0.001656,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.25461, 0.17192, 0.15172, 0.14021, 0.13227, ...",0.0001,1024,8,"[0.001, 0.001]",10,994169,4,../NeuMF_storage/GMF_ckpts_ml_1m_nolf_8/ckpts,Adam
13,0.002318,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.25461, 0.17192, 0.15172, 0.14021, 0.13227, ...",0.0001,512,8,"[0.0001, 0.0001]",10,994169,4,../NeuMF_storage/GMF_ckpts_ml_1m_nolf_8/ckpts,Adam
5,0.002483,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.25461, 0.17192, 0.15172, 0.14021, 0.13227, ...",0.0010,512,8,"[0.0001, 0.0001]",10,994169,4,../NeuMF_storage/GMF_ckpts_ml_1m_nolf_8/ckpts,Adam
29,0.002649,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.54862, 0.5004, 0.5004, 0.5004, 0.5004, 0.50...",0.0001,256,8,"[0.0001, 0.0001]",10,994169,4,../NeuMF_storage/GMF_ckpts_ml_1m_nolf_8/ckpts,Adam
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,0.475497,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.25461, 0.17192, 0.15172, 0.14021, 0.13227, ...",0.0001,512,8,"[1e-05, 1e-05]",10,994169,4,../NeuMF_storage/GMF_ckpts_ml_1m_nolf_8/ckpts,Adam
2,0.476987,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.25461, 0.17192, 0.15172, 0.14021, 0.13227, ...",0.0050,512,8,"[1e-05, 1e-05]",10,994169,4,../NeuMF_storage/GMF_ckpts_ml_1m_nolf_8/ckpts,Adam
22,0.479636,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.40042, 0.35553, 0.31952, 0.29065, 0.28179, ...",0.0005,256,8,"[1e-05, 1e-05]",10,994169,4,../NeuMF_storage/GMF_ckpts_ml_1m_nolf_8/ckpts,Adam
10,0.482616,"[[rank_at, hitcounts, recall, precision, ndcg]...",rank_at hitcounts recall precision ...,"[0.34864, 0.28134, 0.26854, 0.26243, 0.25928, ...",0.0010,128,8,"[1e-05, 1e-05]",10,994169,4,../NeuMF_storage/GMF_ckpts_ml_1m_nolf_8/ckpts,Adam


In [2]:
res.sort_values('test_rec@10')[-2:-1]

NameError: name 'res' is not defined

In [48]:
res.to_pickle(store_path + 'all_GMF_GS')